# LAB GenAI - LLMs - OpenAI Assistant

In [22]:
# Imports
import fitz 
import openai
import requests

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

from dotenv import load_dotenv

## Exercise 1

Create an assistant to answer a topic of your choosing:
 - Upload a file of your interest
 - Add Instructions to the prompt
 - Use the assistant in Playground mode

 https://platform.openai.com/playground/assistants

In [13]:
# Read OpenAI API key from file
with open("API_Key.txt", "r") as file:
    openai_api_key = file.read().strip()  # Remove extra spaces/newlines

# Set the API key as an environment variable
import os
os.environ["OPENAI_API_KEY"] = openai_api_key  # Required for LangChain

# Now, initialize OpenAI embeddings
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)  # Pass explicitly

# Step 1: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# Step 2: Load PDF and split text into chunks
pdf_path = "Electronics for Dummies.pdf"
text_data = extract_text_from_pdf(pdf_path)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Split into 1000-character chunks
    chunk_overlap=100  # Ensure context between chunks
)
chunks = text_splitter.split_text(text_data)

# Step 3: Create vector store (ChromaDB) using OpenAI embeddings
embedding = OpenAIEmbeddings()
vector_store = Chroma.from_texts(chunks, embedding)

# Step 4: Set up Retrieval-Augmented Generation (RAG)
retriever = vector_store.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4", temperature=0),
    retriever=retriever
)

# Step 5: User Query
user_query = "What is Ohm's Law?"
response = qa_chain.run(user_query)

print(response)

C:\Users\dylan\AppData\Local\Temp\ipykernel_20784\1030107297.py:45: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(model_name="gpt-4", temperature=0),
C:\Users\dylan\AppData\Local\Temp\ipykernel_20784\1030107297.py:51: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(user_query)


Ohm’s Law is an equation that allows you to calculate the relationship between voltage, current, and resistance in an electrical circuit. It states that the voltage equals current multiplied by resistance, or in standard mathematical notation V = I x R. This means if you know any two of the three values in the equation, you can calculate the third.


## Exercise 2

Talk to your assistant via the API

https://platform.openai.com/docs/assistants/overview

In [18]:
# Load API Key from .env
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

# Step 1: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# Load PDF and process it
pdf_path = "Electronics for Dummies.pdf"
text_data = extract_text_from_pdf(pdf_path)

# Step 2: Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_text(text_data)

# Step 3: Create vector store with OpenAI embeddings
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)
vector_store = Chroma.from_texts(chunks, embedding)

# Step 4: Set up Conversational Retrieval Chain (keeps context)
retriever = vector_store.as_retriever()
qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(model_name="gpt-4", temperature=0),
    retriever=retriever
)

# Step 5: Conversation loop
chat_history = []  # Stores previous interactions
print("Electronics Assistant (type 'exit' to stop)")
while True:
    user_query = input("\nYou: ")
    if user_query.lower() == "exit":
        print("Goodbye!")
        break
    
    response = qa_chain({"question": user_query, "chat_history": chat_history})
    chat_history.append((user_query, response["answer"]))  # Keep history
    
    print("\nAssistant:", response["answer"])

Electronics Assistant (type 'exit' to stop)



You:  Hello, what is a transistor?


C:\Users\dylan\AppData\Local\Temp\ipykernel_20784\1288566343.py:41: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"question": user_query, "chat_history": chat_history})



Assistant: A transistor is a device that controls the flow of electric current by opening and closing a kind of valve within it. It can be used as either a switch or an amplifier. A transistor has three leads: Base, emitter, and collector. When used as a switch, the base lead of the transistor works like the toggle on a mechanical switch. Without transistors, many modern electronic devices like radios, cell phones, and computers would be much larger.



You:  exit


Goodbye!


## Exercise 3

Create an assistant that will call a weather API, given the user's answer and return the proper answer.

See the documentation of the weather API here: https://open-meteo.com/en/docs

In [20]:
import requests

def get_weather_forecast(latitude, longitude):
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m"
    }
    response = requests.get(base_url, params=params)
    return response.json()

# Example usage:
forecast = get_weather_forecast(52.52, 13.41)
print(forecast)

{'latitude': 52.52, 'longitude': 13.419998, 'generationtime_ms': 0.06008148193359375, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 38.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2025-02-04T00:00', '2025-02-04T01:00', '2025-02-04T02:00', '2025-02-04T03:00', '2025-02-04T04:00', '2025-02-04T05:00', '2025-02-04T06:00', '2025-02-04T07:00', '2025-02-04T08:00', '2025-02-04T09:00', '2025-02-04T10:00', '2025-02-04T11:00', '2025-02-04T12:00', '2025-02-04T13:00', '2025-02-04T14:00', '2025-02-04T15:00', '2025-02-04T16:00', '2025-02-04T17:00', '2025-02-04T18:00', '2025-02-04T19:00', '2025-02-04T20:00', '2025-02-04T21:00', '2025-02-04T22:00', '2025-02-04T23:00', '2025-02-05T00:00', '2025-02-05T01:00', '2025-02-05T02:00', '2025-02-05T03:00', '2025-02-05T04:00', '2025-02-05T05:00', '2025-02-05T06:00', '2025-02-05T07:00', '2025-02-05T08:00', '2025-02-05T09:00', '2025-02-05T10:00', '2025-02-05T11:00', '2025-02-05T12:00'

In [26]:
print("API Key:", weather_api_key)


API Key: None


In [30]:
# Function to get weather info using Open-Meteo
def get_weather(city):
    # Open-Meteo coordinates
    city_coords = {
        "Washington": {"lat": 38.9072, "lon": -77.0369},
        "Denver": {"lat": 39.7392, "lon": -104.9903},
        "Paris": {"lat": 48.8566, "lon": 2.3522},
        "London": {"lat": 51.5074, "lon": -0.1278},
    }
    
    # Check if city is in our predefined coordinates
    if city not in city_coords:
        return "City not found in database."
    
    coords = city_coords[city]
    lat, lon = coords["lat"], coords["lon"]
    
    # Open-Meteo API URL
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "current_weather": "true"
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        weather_code = data["current_weather"]["weathercode"]
        temperature = data["current_weather"]["temperature"]
        
        # Mapping weather codes to human-readable descriptions
        weather_conditions = {
            0: "Clear sky",
            1: "Mainly clear",
            2: "Partly cloudy",
            3: "Cloudy",
            4: "Overcast",
            5: "Showers",
            6: "Heavy showers",
            7: "Thunderstorms",
            8: "Heavy thunderstorms",
            9: "Snow"
        }
        
        weather = weather_conditions.get(weather_code, "Unknown weather condition")
        return f"The weather in {city} is {weather} with a temperature of {temperature}°C."
    else:
        return "Error: Couldn't retrieve weather data."

# Simple interaction
while True:
    city = input("\nEnter a city name (or type 'exit' to stop): ")
    if city.lower() == "exit":
        print("Goodbye!")
        break
    print(get_weather(city))


Enter a city name (or type 'exit' to stop):  Washington


The weather in Washington is Clear sky with a temperature of 13.8°C.



Enter a city name (or type 'exit' to stop):  exit


Goodbye!


### If you want to, there is a hint here:

OpenAI Chatbots / Assistants have a way to respond in json format. 

Explore the function calling functionality